# IterativeEikonal
Approximate the viscosity solution of the  Eikonal equation 
$$\begin{cases} \Vert \nabla_G W(g) \Vert = 1, & g \in G \setminus S, \\
W(g) = 0, & g \in S \subset G. \end{cases}$$
This method is based on the paper ["A PDE Approach to Data-Driven Sub-Riemannian Geodesics in $SE(2)$" (2015) by E. J. Bekkers, R. Duits, A. Mashtakov, and G. R. Sanguinetti](https://doi.org/10.1137/15M1018460).

In [ ]:
import iterativeeikonal as eik
import taichi as ti
ti.init(arch=ti.gpu, debug=False)
import numpy as np
import scipy as sp
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
%matplotlib widget

## $\mathbb{R}^2$

### Flat space
In fact, we will solve the Eikonal equation on some subset $G \subset \mathbb{R}^2$, namely $G = [-1, 1] \times [-1, 1]$. The source set will consist of a single point: $S = \{(0, 0)\}$. Then we know that the viscosity solution is simply the Euclidean norm. 

There are numerous ways to numerically solve the Eikonal equation. We could use [Fast Marching](https://en.wikipedia.org/wiki/Fast_marching_method). This is an efficient method, but it requires a lot of work to make it compatible with non-Euclidean domains. 

In [ ]:
def fast_marching_R2(f):
    """
    Naive implementation of fast marching to solve Eikonal equation on R2 with 
    cost `f`."""
    N = f.shape[0]
    f = pad_array(f, 1.)
    dxy = 2. / N
    point_stages = 2 * np.ones((N, N), dtype=int)
    W = np.full(shape=(N, N), fill_value=100.)
    point_stages = pad_array(point_stages, 0)
    W = pad_array(W, 100.)
    i_0, j_0 = ((N + 1) // 2, (N + 1) // 2)
    point_stages[i_0, j_0] = 0
    W[i_0, j_0] = 0
    i_star, j_star = i_0, j_0
    while np.any(point_stages != 0):
        update_neighbours(i_star, j_star, point_stages, W, dxy, f)
        Trial = point_stages == 1
        index = np.argmin(np.where(Trial, W, np.inf))
        i_star, j_star = np.unravel_index(index, W.shape)
        point_stages[i_star, j_star] = 0
    return unpad_array(W)

def pad_array(array, fill_value):
    padded_shape = (array.shape[0] + 2, array.shape[1] + 2)
    padded_array = np.ones(padded_shape, dtype=array.dtype) * fill_value
    padded_array[1:-1, 1:-1] = array
    return padded_array

def unpad_array(padded_array):
    return padded_array[1:-1, 1:-1]

def update_neighbours(i_star, j_star, point_stages, W, dxy, f):
    neighbours = sees_point(i_star, j_star)
    for i, j in neighbours:
        if point_stages[i, j] != 0:
            point_stages[i, j] = 1
            update_W(i, j, W, dxy, f)

def update_W(i, j, W, dxy, f):
    Wx, Wy = gradient_W(i, j, W)
    if np.abs(Wx - Wy) >= dxy:
        W[i, j] = np.min((Wx + dxy / f[i, j], Wy + dxy / f[i, j]))
    else:
        W[i, j] = (Wx + Wy + np.sqrt((Wx + Wy) ** 2 - 2 * (Wx ** 2 + Wy ** 2 - (dxy / f[i, j]) ** 2))) / 2

def gradient_W(i, j, W):
    Wx = min(W[i + 1, j], W[i - 1, j])
    Wy = min(W[i, j + 1], W[i, j - 1])
    return Wx, Wy

def sees_point(i_star, j_star):
    return ((i_star + 1, j_star), (i_star - 1, j_star), (i_star, j_star + 1), (i_star, j_star - 1))

We can alternatively apply an iterative method, developed by [Bekkers et al.](https://doi.org/10.1137/15M1018460)

In [ ]:
def iterative_method_flat_R2(N, n):
    """
    Discretise [-1, 1] x [-1, 1] into `N` points in each direction, and apply 
    the iterative solution method `n` times.
    """
    dxy = 2. / (N + 1)
    eps = dxy / 4

    W = get_initial_W(N)
    boundarypoints, boundaryvalues = get_boundary_conditions(N)

    eik.cleanarrays.apply_boundary_conditions(W, boundarypoints, boundaryvalues)

    dx_forward, dx_backward, dy_forward, dy_backward, abs_dx, abs_dy = get_initial_derivatives(W)
    for _ in tqdm(range(n)):
        step_W(W, dx_forward, dx_backward, dy_forward, dy_backward, abs_dx, abs_dy, dxy, eps)
        eik.cleanarrays.apply_boundary_conditions(W, boundarypoints, boundaryvalues)

    W_np = W.to_numpy()
    return eik.cleanarrays.unpad_array(W_np)

def get_initial_W(N, initial_condition=100.):
    W_unpadded = np.full(shape=(N, N), fill_value=initial_condition)
    W_np = eik.cleanarrays.pad_array(W_unpadded, pad_value=initial_condition, pad_shape=1)
    W = ti.field(dtype=ti.f32, shape=W_np.shape)
    W.from_numpy(W_np)
    return W

def get_boundary_conditions(N):
    i_0, j_0 = (N + 1) // 2, (N + 1) // 2
    boundarypoints_np = np.array([[i_0, j_0]], dtype=int)
    boundaryvalues_np = np.array([0.], dtype=float)
    boundarypoints = ti.Vector.field(n=2, dtype=ti.i32, shape=1)
    boundarypoints.from_numpy(boundarypoints_np)
    boundaryvalues = ti.field(shape=1, dtype=ti.f32)
    boundaryvalues.from_numpy(boundaryvalues_np)
    return boundarypoints, boundaryvalues

def get_initial_derivatives(W):
    dx_forward = ti.field(dtype=ti.f32, shape=W.shape)
    dx_backward = ti.field(dtype=ti.f32, shape=W.shape)
    dy_forward = ti.field(dtype=ti.f32, shape=W.shape)
    dy_backward = ti.field(dtype=ti.f32, shape=W.shape)
    abs_dx = ti.field(dtype=ti.f32, shape=W.shape)
    abs_dy = ti.field(dtype=ti.f32, shape=W.shape)
    return dx_forward, dx_backward, dy_forward, dy_backward, abs_dx, abs_dy

@ti.kernel
def step_W(
    W:ti.template(), 
    dx_forward: ti.template(), 
    dx_backward: ti.template(), 
    dy_forward: ti.template(), 
    dy_backward: ti.template(), 
    abs_dx: ti.template(), 
    abs_dy: ti.template(), 
    dxy: ti.f32, 
    eps: ti.f32
):
    eik.derivativesR2.abs_derivatives(W, dxy, dx_forward, dx_backward, dy_forward, dy_backward, abs_dx, abs_dy)
    for I in ti.grouped(W):
        W[I] += (1 - ti.math.sqrt(abs_dx[I] ** 2 + abs_dy[I] ** 2)) * eps

In [ ]:
N = 51
n = 250
xs, ys = np.meshgrid(np.linspace(-1, 1, N), np.linspace(-1, 1, N))
W_exact = np.sqrt(xs ** 2 + ys ** 2)
W_iterative_big = iterative_method_flat_R2(N, n)
f = np.ones((N, N), dtype=float)
W_fast_marching = fast_marching_R2(f)

In [ ]:

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 5))
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.resizable = False
contour = ax.contour(xs, ys, W_fast_marching, linestyles="dotted")
contour = ax.contour(xs, ys, W_iterative_big, linestyles="dashed")
ax.contour(xs, ys, W_exact)
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.set_xlabel("x")
ax.set_ylabel("y")
fig.colorbar(contour, label="W(x, y)");

### Retinal image
Computing the cost function on flat $\mathbb{R}^2$ is not very impressive. We would like to be able to do it also when the space is not flat, for instance in the case of a retinal image. To track the vessels in a retinal image, it makes sense to assign a low cost to those areas of the image that are vessels, and a high cost to those that aren't. We must therefore first compute such a cost function.

To do this, we use Frangi vesselness filtering.

#### Frangi vesselness filtering
Let's first load in the image.

In [ ]:
ds = 8
retinal_image = Image.open("E46_OD_best.tif")
width, height = retinal_image.size
retinal_image_gray_ds = retinal_image.resize((width // ds, height // ds)).convert("L")
retinal_array_unnormalised = np.array(retinal_image_gray_ds).astype(np.float64)
retinal_array = retinal_array_unnormalised / retinal_array_unnormalised.max()

We will already define the start and the end of our geodesic (`source_point` and `target_point`, respectively).

In [ ]:
source_point = (246, 302) # "y", "x" so row, column.
target_point = (211, 118)
i_min, i_max = 0, retinal_array.shape[0] -1
j_min, j_max = 0, retinal_array.shape[0] -1
xs, ys = np.meshgrid(np.arange(i_min, i_max + 1), np.arange(j_min, j_max + 1))

We can now perform the Frangi filtering. The values for the scales, as well as the Frangi filter parameters $\alpha$, $\gamma$, and $\varepsilon$ were taken from the Mathematica notebook "Code A - Vesselness in SE(2).nb".

In [ ]:
scales = (np.array((2, 3, 4, 5), dtype=float))
vesselness = eik.costfunctions.multiscale_frangi_filter_R2(-retinal_array, scales, α=0.3, γ=3/4, ε=0.3)
mask = (retinal_array > 0) # Remove boundary
vesselness *= sp.ndimage.binary_erosion(mask, iterations=int(np.ceil(scales.max() * 2)))
print(f"Before rescaling, vesselness is in [{vesselness.min()}, {vesselness.max()}].")
vesselness /= vesselness.max()

In [ ]:
image_vesselness = eik.cleanarrays.convert_array_to_image(vesselness)
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10))
ax.imshow(image_vesselness, cmap="gray", origin="upper")
ax.scatter(xs[source_point], ys[source_point], label="Source", marker=".")
ax.scatter(xs[target_point], ys[target_point], label="Target", marker=".")
ax.set_xlabel("j")
ax.set_ylabel("i")
ax.set_xlim(j_min, j_max)
ax.set_ylim(i_max, i_min)
ax.legend();

With these parameters, the vesselness score produced by the Frangi filter looks similar to the one in the Mathematica notebook "code A - Vesselness in SE(2).nb". However, the normalisation is not the same.

#### Cost function
Given the vesselness function, we compute the cost function as
$$\mathrm{cost}_{\lambda, p}(x, y) = \frac{1}{1 + \lambda \cdot \vert \mathrm{vesselness} \vert^p}$$
We choose $p = 2$, like in the Mathematica notebook, but $\lambda = 100$ instead of $\lambda = 1000$. This produces a cost function that looks more similar (probably due to the difference in normalisation).

In [ ]:
cost = eik.costfunctions.cost_function(vesselness, 100, 2)

In [ ]:
image_cost = eik.cleanarrays.convert_array_to_image(cost)
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10))
ax.imshow(image_cost, cmap="gray", origin="upper")
ax.scatter(xs[source_point], ys[source_point], label="Source", marker=".")
ax.scatter(xs[target_point], ys[target_point], label="Target", marker=".")
ax.set_xlabel("j")
ax.set_ylabel("i")
ax.set_xlim(j_min, j_max)
ax.set_ylim(i_max, i_min)
ax.legend();

#### Distance map
We are now able to compute the distance map, which is the viscosity solution of the Eikonal equation
$$\begin{cases} \Vert \nabla_{\mathrm{cost}} W(x, y) \Vert = 1, & (x, y) \in \mathbb{R}^2 \setminus \{(x_0, y_0)\}, \\
W(x, y) = 0, & (x, y) = (x_0, y_0), \end{cases}$$
where $\nabla_C$ is a datadriven derivative:
$$\Vert \nabla_{\mathrm{cost}} W(x, y) \Vert = \sqrt{\mathrm{cost}_{\lambda, p}^{-2}(x, y) (\vert \partial_x W(x, y) \vert^2 + \vert \partial_y W(x, y) \vert^2)}.$$
The quality of the approximation spreads out from the source point. Hence, if we are interested only in the distance map to points that are not that far away (in terms of the true distance map), then we do not have to perform as many iterations. We terminate the iteration when the value of the approximate distance map changes less than a certain tolerance ($10^{-5}$) after a step.

In [ ]:
W_iterative = eik.eikonal_solver_R2(cost, source_point, target_point)
# W_fast_marching = fast_marching_R2(cost) # Too slow...

If all is well, then the distance map should be $0$ at the source point, and vary nicely around the source_point.

In [ ]:
W_iterative[source_point[0]-3:source_point[0]+2, source_point[1]-3:source_point[1]+2]

We can now visualise the distance map with a contour plot.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 10))
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.resizable = False
ax.imshow(cost, cmap="gray", origin="upper")
max_distance = np.round(W_iterative[target_point[1], target_point[0]] * 1.1)
contour = ax.contour(xs, ys, W_iterative, levels=np.linspace(0., max_distance, 5))
ax.scatter(xs[source_point], ys[source_point], label="Source", marker=".")
ax.scatter(xs[target_point], ys[target_point], label="Target", marker=".")
ax.set_xlabel("j")
ax.set_ylabel("i")
ax.set_xlim(j_min, j_max)
ax.set_ylim(i_max, i_min)
ax.legend()
fig.colorbar(contour, label="W[i, j]");

### Geodesic tracking
Now that we have the distance map (from a certain source point), we are able to compute the geodesics between the source point and any other point $(x^*, y^*)$ using backtracking:
$$\begin{dcases} \dot{\gamma}(t) = -\left(\frac{\partial_x W(\gamma(t))}{\mathrm{cost}_{\lambda, p}^2(\gamma(t))}, \frac{\partial_y W(\gamma(t))}{\mathrm{cost}_{\lambda, p}^2(\gamma(t))}\right), & t > 0, \\
\gamma(0) = (x^*, y^*). & \end{dcases}$$
We can numerically solve this using a Forward Euler discretisation:
$$\begin{dcases} \gamma_{n + 1} = \gamma_n - \Delta t \left(\frac{\partial_x W(\gamma_n)}{\mathrm{cost}_{\lambda, p}^2(\gamma_n)}, \frac{\partial_y W(\gamma_n)}{\mathrm{cost}_{\lambda, p}^2(\gamma_n)}\right), & n > 0, \\
\gamma_0 = (x^*, y^*). & \end{dcases}$$

In [ ]:
γ_ci = eik.geodesic_back_tracking_R2(W_iterative, cost, source_point, target_point, β=0.5, n_max=5000)
γ = eik.convert_continuous_indices_to_real_space_R2(γ_ci, xs, ys)

We can see from the visualisation below that the geodesic follows the vessel rather well.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(14, 5))
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.resizable = False

ax[0].imshow(retinal_image_gray_ds, cmap="gray")
contour = ax[0].contour(xs, ys, W_iterative, levels=np.linspace(0., max_distance, 5))
ax[0].plot(γ[:, 0], γ[:, 1], label="geodesic", color="red")
ax[0].scatter(xs[source_point], ys[source_point], label="Source", marker=".")
ax[0].scatter(xs[target_point], ys[target_point], label="Target", marker=".")
ax[0].set_xlabel("j")
ax[0].set_ylabel("i")
ax[0].set_xlim(j_min, j_max)
ax[0].set_ylim(i_max, i_min)
ax[0].legend()
fig.colorbar(contour, label="W[i, j]")

ax[1].imshow(retinal_image_gray_ds, cmap="gray")
contour = ax[1].contour(xs, ys, W_iterative, levels=np.linspace(0., max_distance, 5))
ax[1].plot(γ[:-1, 0], γ[:-1, 1], label="geodesic", color="red")
ax[1].scatter(xs[source_point], ys[source_point], label="Source", marker=".")
ax[1].scatter(xs[target_point], ys[target_point], label="Target", marker=".")
ax[1].set_xlabel("j")
ax[1].set_ylabel("i")
ax[1].set_xlim(100, 320)
ax[1].set_ylim(340, 120)
ax[1].legend()
fig.colorbar(contour, label="W[i, j]");